In [1]:
import os

In [2]:
%pwd
os.chdir("..")

In [3]:
%pwd

'c:\\Users\\user\\Desktop\\End-to-End-ML-project-MLflow'

In [18]:
# entity

from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [19]:
# configuration manager
from my_project.constants import *
from my_project.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
        schema_filepath: Path = SCHEMA_FILE_PATH,
    ):
        # Load configs
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)  # dict (safe for schema)

        # Ensure artifacts root exists
        create_directories([Path(self.config.artifacts_root)])
    
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([Path(config.root_dir)])
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path)
        )
        return data_transformation_config

In [ ]:
# components of data transformation
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from my_project import logger


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def initiate_data_transformation(self):
        logger.info("Data Transformation started")
        
        # Load data
        df = pd.read_csv(self.config.data_path)

        # Split into train and test
        train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
        
        # Print or log shapes
        print("Train shape:", train_df.shape)
        print("Test shape:", test_df.shape)
        logger.info(f"Train shape: {train_df.shape}, Test shape: {test_df.shape}")
        
        # Define file paths
        train_path = os.path.join(self.config.root_dir, "train.csv")
        test_path = os.path.join(self.config.root_dir, "test.csv")

        # Save transformed data
        train_df.to_csv(train_path, index=False)
        test_df.to_csv(test_path, index=False)
        
        logger.info(f"Transformed data saved at {self.config.root_dir}")
        return train_path, test_path


In [ ]:
#pipeline execution
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    print(data_transformation_config)
    
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.initiate_data_transformation()
except Exception as e:
    logger.error(f"Error in Data Transformation: {e}")
    raise


2025-08-28 11:35:06,262 - my_project - INFO - YAML file 'config\config.yaml' read successfully.
2025-08-28 11:35:06,269 - my_project - WARNING - YAML file params.yaml is empty.
2025-08-28 11:35:06,271 - my_project - INFO - YAML file 'params.yaml' read successfully.
2025-08-28 11:35:06,283 - my_project - INFO - YAML file 'schema.yaml' read successfully.
2025-08-28 11:35:06,292 - my_project - INFO - Directory created: 'artifacts'
2025-08-28 11:35:06,299 - my_project - INFO - Directory created: 'artifacts\data_transformation'
DataTransformationConfig(root_dir=WindowsPath('artifacts/data_transformation'), data_path=WindowsPath('artifacts/data_validation/cleaned_data.csv'))
2025-08-28 11:35:06,304 - my_project - INFO - Data Transformation started
Train shape: (943, 12)
Test shape: (236, 12)
2025-08-28 11:35:06,323 - my_project - INFO - Train shape: (943, 12), Test shape: (236, 12)
2025-08-28 11:35:06,382 - my_project - INFO - Transformed data saved at artifacts\data_transformation
